# Práctica 2: Análisis de población vs factores de desarrollo

**C03 : Visualización Científica y Narrativas**

**RAUGM 2021: Geociencias e inclusión**

**Luis Miguel de la Cruz Salas**

This notebook by Luis M. de la Cruz Salas is licensed under <a href="https://creativecommons.org/licenses/by-nc-nd/4.0?ref=chooser-v1">Attribution-NonCommercial-NoDerivatives 4.0 International</a>

Realizar un análisis complementario del FR (Fertility Rate) mundial comparando con otros factores de desarrollo.

Usaremos la misma información de la práctica 1, así como los scripts que se desarrollaron en ella para continuar con esta práctica 2.

## Importar las bibliotecas 
Incluimos las bibliotecas necesarias para la lectura de datos y para la visualización.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

En esta práctica usaremos un estilo de fondo obscuro con texto y gráficos claros. Para ello haremos lo siguiente:

## Lectura de datos de FR
Usamos la misma información de la práctica 1.

In [ ]:
FR = pd.read_csv('../01/UNdata_Export_20211021_200853345.zip')

# Se agrupa por país 
paises = FR.groupby('Country or Area')

In [ ]:
FR

## Funciones para graficación
Ponemos todos los scripts de la práctica 1, con algunas modificacines, en funciones como sigue:

In [ ]:
from math import ceil

def maxminTicks(FR):
    """
    Calcula el máximo y el mínimo de todos los países y los yticks.
    
    Parameters
    ----------
    paises : DataFrameGroupBy
        Dataframe generado por GroupBy con la información de los países.
        
    Returns
    -------
    p_max, y_max, p_min, y_min, yticks
        El país con el máximo valor, el valor máximo, la lista para los yticks,
        el país con el valor mínimo y el valor mínimo.
    """    
    # Se obtiene el valor máximo
    y_max = FR['Value'].max() 

    # Extrae el nombre del país con el valor máximo
    p_max = FR[FR['Value'] == y_max].iloc[0][0]

    # Se obtiene el valor mínimo
    y_min = FR['Value'].min() 

    # Extrae el nombre del país con el valor mínimo
    p_min = FR[FR['Value'] == y_min].iloc[0][0]

    # Se generan los yticks
    yticks = [i for i in range(0,ceil(y_max)+1)]

    return p_max, y_max, p_min, y_min, yticks

In [ ]:
def inicializaGrafica(y_maximo, yticks):
    """
    Inicializa algunos parámetros de la figura (el canvas).
    
    Parameters
    ----------
    y_maximo : int
        Valor máximo para el eje y.
    
    yticks : list
        Lista de valores para los ticks en el eje y.
    """
    fig = plt.figure(figsize=(10,10)) 
    plt.xticks(rotation=70, fontsize=10)
    plt.xlim(-2,14,-1)
    plt.gca().invert_xaxis()
    plt.ylim(0,y_maximo)   
    plt.yticks(yticks)     
    plt.grid(ls='--', lw=0.5)

    # Información adicional y títulos
    plt.title('Promedio de número de hijos por mujer', loc='left', fontsize=12)
    plt.title('fuente: http://data.un.org', loc='right', fontstyle='italic', fontsize=10)
    plt.suptitle('Evolución del FR (Fertility Rate)', y = 0.94, fontsize=14)

    # Se eliminan las líneas del marco de la gráfica
    ejes = plt.gca()
    ejes.spines['right'].set_visible(False)
    ejes.spines['top'].set_visible(False)
    ejes.spines['left'].set_visible(False)
    ejes.spines['bottom'].set_visible(False)
    
    # Modificamos algunos parámetros de los ticks en el eje y
    ejes.tick_params(axis='y', width=1, length=25)
    
    # Realizamos algunas anotaciones sobre el gráfico base
    plt.annotate('Nivel de \n reemplazo: \n promedio = 2.1', 
                 xy=(11.5, 2.095), xytext=(11.5, 1.0),
                 bbox=dict(boxstyle='round', facecolor='gray', edgecolor='black', alpha=0.1, linewidth=0.75),
                 arrowprops=dict(arrowstyle='->', facecolor='black', edgecolor='black'),
                 fontsize=10, color='black', horizontalalignment='center')
    
    plt.text(2, 8.25, 'Cada línea representa la \n evolución del promedio  \n de hijos por mujer en un país', 
             transform=plt.gca().transData, horizontalalignment='center', color='black',
             bbox=dict(boxstyle='round', facecolor='gray', edgecolor='black', alpha=0.1, linewidth=0.75))  


In [ ]:
def graficaFR(paises, parametros={}):
    """
    Realiza la gráfica de todos los países.
    
    Parameters
    ----------
    paises : DataFrameGroupBy
        Dataframe generado por GroupBy con la información de los países.
    
    parametros : dict
        Parámetros para generar la gráfica.
    """
    for p in paises.groups.keys():
        pais = paises.get_group(p)
        plt.plot(pais['Year(s)'], pais['Value'], **parametros) 
        
    # Al final de todas las gráficas ponemos la del nivel de reemplazo 
    plt.plot([-1,14],[2.1,2.1], 'k--', lw=1.0, zorder=1000)

In [ ]:
def graficaFR_Pais(paises, p, parametros={}, par_glow = None):
    """
    Realiza la gráfica de un solo país con realce para fondo negro.
    
    Parameters
    ----------
    paises : DataFrameGroupBy
        Dataframe generado por GroupBy con la información de los países.
    
    parametros : dict
        Parámetros para generar la gráfica.
    
    par_glow : dict
        Diccionario con los parámetros para resaltar las curvas con un "halo" 
        a su alrededor. Cuando par_glow = None (defalt) no se hace nada. En otro
        caso es conveniente pasar la transparencia y el color. Esta
        curva se dibuja por detrás de la curva principal, con un ancho mayor y el
        color definido
        Ejemplo:
                par_glow = {'alpha':0.4, 'c':'yellow'}
    """
    pais = paises.get_group(p)

    if par_glow:
    # Se grafica una curva con una línea 3 veces más ancha que la original
    # y transparente para resaltarla. 
        plt.plot(pais['Year(s)'], pais['Value'], lw=parametros['lw']*3, **par_glow)

    # Se grafica la curva del país con los parámetros necesarios.
    line = plt.plot(pais['Year(s)'], pais['Value'], **parametros)

    # Ponemos un texto al final de la curva para mostrar el 
    # nombre del país y el valor final de fertilidad
    pais_val = pais['Value'].iloc[0]
    plt.text(x = 0, y = pais_val, 
             s = ' {} {:1.2f}'.format(p, pais_val), 
             c = line[0].get_color(), weight = 'bold')
    
    # Ponemos el valor inicial de fertilidad al principio de la curva.
    pais_val = pais['Value'].iloc[-1] 
    plt.text(x = 13.75, y = pais_val, 
             s = '{:1.2f} '.format(pais_val), 
             c = line[0].get_color(), weight = 'bold')

## Visualización del *canvas* 
Calculamos el máximo en el eje $y$ y los `yticks` y posteriormente hacemos la gráfica base.

In [ ]:
p_max, y_max, p_min, y_min, yticks = maxminTicks(FR)
print('Máximo = {}, \t País : {}'.format(y_max, p_max))
print('Mínimo = {}, \t País : {}'.format(y_min, p_min))
print('yticks : {}'.format(yticks))

In [ ]:
# Hacemos la gráfica base
inicializaGrafica(y_max, yticks)
graficaFR(paises, {'lw':0.5, 'c':'#FFFFFF'})

Este es ahora nuestro lienzo listo para agregar información.

## Selección de países con diferente GDP.
Definimos un conjunto de países a ser graficados y sus colores correspondientes. Esta elección se basa en el  GDP (Gross Domestic Product), véase la siguiente figura:

<img src="../../Figuras/PerCapitaIncome.png"  style="width: 800px;"/>


De acuerdo con la gráfica anterior, vamos a elegir países con diferente GDP y diferente zona geográfica, en este caso serán: México, USA, Japón, Alemania, Egipto, Argentina, Nigeria. Además agregamos a los países que hayan tenido el máximo y el mínimo de FR en el rango de tiempo estudiado. 

In [ ]:
paises_colores = {
    'Mexico'                   : 'C0',  
    'United States of America' : 'C1',
    'Japan'                    : 'C2',  
    'Germany'                  : 'C3',  
    'Egypt'                    : 'C4',
    'Argentina'                : 'C5',
    'Nigeria'                  : 'C6',
    p_max                      : 'red',      # País con el valor máximo
    p_min                      : 'blue',     # País con el valor mínimo
    'World'                    : 'black'     # Datos promedio mundiales
}

### Visualización de los países
Realizamos las gráficas para estos países.

In [ ]:
# Hacemos la gráfica base
inicializaGrafica(y_max, yticks)
graficaFR(paises, {'lw':0.5, 'c':'lightgrey'})

# Hacemos la gráfica para los países definidos antes para 
# hacer la comparación entre ellos.
for p, c in paises_colores.items():
    par = {'lw':3.0, 'c':c}
    graficaFR_Pais(paises, p, par)

plt.savefig('FR01.pdf')

Si puso atención, en la definición de la función `graficaFR_Pais()` hay un parámetro
para resaltar las gráficas. Usaremos ese parámetro como sigue:

In [ ]:
# Hacemos la gráfica base
inicializaGrafica(y_max, yticks)
graficaFR(paises, {'lw':0.5, 'c':'lightgrey'})

# Hacemos la gráfica para los países definidos antes para hacer la comparación entre ellos.
for p, c in paises_colores.items():
    par = {'lw':1.5, 'c':c, 'marker':'.'}
    par_glow = {'alpha':0.25, 'c':c}         # Parámetros para poner "brillo" a las curvas
    graficaFR_Pais(paises, p, par, par_glow) # Pasamos los parámetros a la función
    
plt.savefig('FR02.pdf')

En esta visualización intentamos graficar países de los diferentes continentes y con un GDP (Gross Domestic Product, en miles de millones de dolares) diferente: USA (20,544), Japon (4,971), Alemania (3,948), Mexico (1,221), Argentina (519), Egipto (250), Macao SAR (55), Yemen (27), Nigeria (9). (Información tomada de https://data.worldbank.org/country). Observamos que no necesariamente entre menos GDP mayor fertilidad, véase por ejemplo el caso de Macao SAR. Sin embargo, en general si se observa que los países con mayor GDP tienden a bajar el número de hijos por mujer por debajo del NR. Los tres países que en esta visualización se ven com mayor fertilidad (Egipto, Yemen y Nigeria) tienen un GPD bajo, comparado con los otros países, pero todos han reducido su porcentaje de fertilidad. Nigeria es un caso particular, pues solo ha bajado de 6.35 en 1955 a 5.42 en 2020.  

## Datos de GDP por país
Vamos ahora a complementar esta visualización con información del GPD per capita en cada país.

Obtenemos la información de los ingresos por persona en cada país de http://data.un.org haciendo la búsqueda de *per capita income* como se muestra en la siguiente figura:

<img src="../../Figuras/PerCapita.png"  style="width: 800px;"/>

**Observación** : los datos son solo para el año 2010, que es el año en el que vamos a realizar la comparación. Por esta razón tomaremos los datos de la década 2005-2010 para el caso del FR (esta información se encuentra en el lugar 2 del arreglo de cada país, véanse las gráficas anteriores y recuérdese que el eje $x$ fue invertido).

In [ ]:
# Leemos el archivo que obtiene en un DataFrame
per_capita_income = pd.read_csv('UNdata_Export_20211023_002155840.zip')
pd.set_option('display.max_rows', None) # Para poder ver todo el dataframe
per_capita_income

In [ ]:
pd.set_option('display.max_rows', 15) # Regreso a un número limitado de renglones por despliegue de dataframe

### Tratamiento de los datos.
Observamos que el número de países listados en los dos dataframes, el de fertilidad y el de ingresos, es diferente, así que necesitamos ajustarlos para poder hacer una comparación entre ellos.

In [ ]:
# Se agrupa por país la información de los ingresos 
# (para acceder más fácil y de manera similar a como se 
# hace con la información del FR)
ingreso_pais = per_capita_income.groupby('Country or Area')

In [ ]:
# Checamos la longitud de cada agrupación para ver si coinciden
print(len(paises.groups.keys()), len(per_capita_income)) 

In [ ]:
# Se obtiene la lista de países a comparar:
lista_paises_final = []

# Hacemos el recorrido usando el DataFrame de mayor longitud (paises)
for p in paises.groups.keys():     
    try:                             # Aseguramos que se tiene la misma info
        ingreso_pais.get_group(p)    # en cada DataFrame.
        lista_paises_final.append(p) # Si se tuvo éxito entonces se agrega el país a la lista final.
    except KeyError: # Captura de la excepción de tipo KeyError
        continue     # Lo único que hacemos es saltarnos el país que no está 
                     # en ambos DataFrames y continuar con el siguiente.
        
print('Lista final de países en ambos Dataframes : ', len(lista_paises_final), '\n')
print(lista_paises_final)

In [ ]:
# Revisamos la información del FR uno de los países
# para identificar en qué lugar está la info de 2010
paises.get_group('Mexico')

In [ ]:
ingreso_pais.get_group('Mexico')

In [ ]:
# Ahora creamos dos listas, una para el FR y otra para el PCI (Per Capita Income).
l_FR = []
l_PCI = []
for p in lista_paises_final:
    l_FR.append(paises.get_group(p)['Value'].iloc[2])  # El lugar 2 corresponde a los datos para 2010
    l_PCI.append(ingreso_pais.get_group(p)['Value'].iloc[0]) # Solo se tiene el dato para 2010

print('\nFR (tamaño: {}) \nDatos :\n{}'.format(len(l_FR), l_FR))
print('\nPCI (tamaño: {}) \nDatos :\n{}'.format(len(l_PCI), l_PCI))

**Obsérvese** que se tienen `nan` en algunos lugares de la lista de FR, lo que significa que no se tiene información completa para algunos países. Tomaremos esto en cuenta más adelante.

### Visualización FR vs PCI
Ahora haremos la gráfica de esta información usando puntos en el plano.

In [ ]:
fig = plt.figure(figsize=(10,6)) # Cambiamos el tamaño de la figura

# Graficamos el PCI (eje $y$) en función de la FR (eje $x$)
# Usamos un color gris tenue y con transparencia. En este caso, este
# será nuestro lienzo base para lo demás.
plt.scatter(l_FR, l_PCI, marker='o', color='lightgray', alpha=0.5) 

# Información adicional y títulos
plt.title('fuente: http://data.un.org', loc='right', fontstyle='italic', fontsize=10)
plt.suptitle(' FR vs PCI ', y = 0.94, fontsize=14)
    
plt.grid(ls='--', lw=0.5, c='#AAAAAA')
plt.xlabel('FR [No. de hijos por mujer]')
plt.ylabel('PCI [USD/Año]')

Observamos en la visualización anterior que los puntos se aglomeran en ciertas zonas del espacio. Se puede ver un comportamiento decreciente conforme aumenta el FR. 

Vamos a completar la visualización anterior, resaltando los países que elegimos en el paso 5 (probablemente algunos no aparezcan debido a que ajustamos la información).

In [ ]:
fig = plt.figure(figsize=(10,6)) # Cambiamos el tamaño de la figura

# Graficamos el ingreso (eje $y$) en función de la fertilidad (eje $x$)
# Usamos un color gris tenue y con transparencia. En este caso, esta 
# será nuestro lienzo base para lo demás.
plt.scatter(l_FR, l_PCI, marker='o', color='lightgray', alpha=0.5) 

# Información adicional y títulos
plt.title('fuente: http://data.un.org', loc='right', fontstyle='italic', fontsize=10)
plt.suptitle(' FR vs PCI ', y = 0.94, fontsize=14)
    
plt.grid(ls='--', lw=0.5, c='#AAAAAA')
plt.xlabel('FR [No. de hijos por mujer]')
plt.ylabel('PCI [USD/Año]')

# Gráfica de los países elegidos en el paso 5 usando puntos de colores y resaltados.
for p, c in paises_colores.items():
    try:
        ip = lista_paises_final.index(p)
        plt.scatter(l_FR[ip], l_PCI[ip], 
                    marker='o', s=100,
                    facecolor=c, edgecolor='w', alpha=0.75,
                    zorder=1000, label=p)
    except ValueError:
        continue

plt.legend()

### Ajuste de los datos.
Dada la tendencia que se observa en esta gráfica, vamos a intentar ajustar una curva a los datos.

In [ ]:
# Usaremos en este caso la función curve_fit.
from scipy.optimize import curve_fit

# La función que queremos ajustar: exponencial decreciente.
def func(x, a, b, c):
    return a * np.exp(-b * x) + c

# Vamos a limpiar los datos, pues existen algunos 'nan' en ellos, 
# particularmente en la FR.
xdata = []
ydata = []
for lf, li in zip(l_FR, l_PCI):
    if ~np.isnan(lf):
        xdata.append(lf)
        ydata.append(li)
    
# Con las listas xdata y ydata ya podemos hacer el ajuste
popt, pcov = curve_fit(func, xdata, ydata)
perr = np.sqrt(np.diag(pcov))

In [ ]:
# Graficamos el ajuste
x = np.linspace(1,9,100)
y = func(x, *popt)
plt.plot(x, y, 'r-',label='fit: a={:5.3f}, b={:5.3}, c={:5.3f}'.format(popt[0], popt[1], popt[2]))
plt.legend()

Finalmente graficamos todo junto:

In [ ]:
fig = plt.figure(figsize=(10,6)) # Cambiamos el tamaño de la figura

# Graficamos el ingreso (eje $y$) en función de la fertilidad (eje $x$)
# Usamos un color gris tenue y con transparencia. En este caso, esta 
# será nuestro lienzo base para lo demás.
plt.scatter(l_FR, l_PCI, marker='o', color='#ABABAB', alpha=0.5) 

# Información adicional y títulos
plt.title('fuente: http://data.un.org', loc='right', fontstyle='italic', fontsize=10)
plt.suptitle(' FR vs PCI ', y = 0.94, fontsize=14)
    
plt.grid(ls='--', lw=0.5, c='#AAAAAA')
plt.xlabel('FR [No. de hijos por mujer]')
plt.ylabel('PCI [USD/Año]')

# Gráfica de los países elegido en el paso 5 usando puntos de colores y resaltados.
for p, c in paises_colores.items():
    try:
        ip = lista_paises_final.index(p)
        plt.scatter(l_FR[ip], l_PCI[ip], 
                    marker='o', s=75,
                    facecolor=c, edgecolor='black', alpha=0.75,
                    zorder=1000, label=p)
    except ValueError:
        continue

# Gráfica del ajuste resaltada!
plt.plot(x, y, 'r-', lw=3.0, alpha=0.4)
plt.plot(x, y, 'r-', lw=1.0, label='fit: a={:5.3f}, b={:5.3}, c={:5.3f}'.format(popt[0], popt[1], popt[2]))

plt.legend()
plt.savefig('FR03_ajuste.pdf')

¿Que podría mejorar de esta visualización? (colores, dimensiones, anotaciones, leyendas, ajuste, etc.)

## Datos del nivel en educación.
Vamos ahora realizar una visualización similar pero con información del nivel de educación en cada país.

Obtenemos la información de http://data.un.org haciendo la búsqueda que se muestra en la siguiente figura:

<img src="../../Figuras/Educacion.png"  style="width: 800px;"/>

**Observación** : igual que en el caso del ingreso, los datos son solo para el año 2010, que es el año en el que vamos a realizar la comparación.

In [ ]:
# Leemos el archivo generado 
educacion = pd.read_csv('UNdata_Export_20211023_005512887.zip')
educacion

Observamos que la información viene para 'Female', 'Male' y 'All genders'.

In [ ]:
# Organizamos la información por país y por sexo:
educacion_pais = educacion.groupby(['Reference Area', 'Sex'])

# Podemos por ejemplo revisar la información para México:
gender = 'Female' # Este parámetro permite elegir el sexo para la comparación.
educacion_pais.get_group(('Mexico',gender))

### Tratamiento de los datos

Igual que antes, debemos asegurarnos que tenemos el mismo número de datos.

In [ ]:
lista_paises_final2 = []
for p in paises.groups.keys():
    try:
        e = educacion_pais.get_group((p, gender))
        lista_paises_final2.append(p)
    except KeyError:
        continue
        
print('Lista final de países en ambos Dataframes : ', len(lista_paises_final2), '\n')
print(lista_paises_final2)

In [ ]:
# Ahora creamos dos listas, una para el FR y otra para el nivel educativo
l_FR2 = []
l_ED = []
for p in lista_paises_final2:
    l_FR2.append(paises.get_group(p)['Value'].iloc[2])  # El lugar 2 corresponde a los datos para 2010
    
    e = educacion_pais.get_group((p, gender))
    l_ED.append(e['Observation Value'].iloc[0]) # Solo se tiene el dato para 2010

print('\nFR (tamaño: {}) \nDatos :\n{}'.format(len(l_FR2), l_FR2))
print('\nEducación (tamaño: {}) \nDatos :\n{}'.format(len(l_ED), l_ED))

### Visualización FR vs Educación

In [ ]:
fig = plt.figure(figsize=(10,6)) # Cambiamos el tamaño de la figura

# Graficamos el nivel de eduación (eje $y$) en función de la fertilidad (eje $x$)
plt.scatter(l_FR2, l_ED, marker='o', color='lightgray', alpha=0.5) 

# Información adicional y títulos
plt.title('fuente: http://data.un.org', loc='right', fontstyle='italic', fontsize=10)
plt.suptitle('FR vs Educación (mujer) ', y = 0.94, fontsize=14)
    
plt.grid(ls='--', lw=0.5, c='#AAAAAA')
plt.xlabel('FR [No. de hijos por mujer]')
plt.ylabel('% en tercer nivel educativo ({})'.format(gender))

# Gráfica de los países elegido en el paso 5 usando puntos de colores y resaltados.
for p, c in paises_colores.items():
    try:
        ip = lista_paises_final2.index(p)
        plt.scatter(l_FR[ip], l_ED[ip], 
                    marker='o', s=75,
                    facecolor=c, edgecolor='k', alpha=0.75,
                    zorder=1000, label=p)
    except ValueError:
        continue

plt.legend(loc='upper center')

- ¿Qué se puede decir de esta visualización?
- ¿Cómo se podría mejorar? (colores, dimensiones, anotaciones, leyendas, etc)
- ¿Podría realizar un ajuste de una curva a estos datos?
- ¿Cree que los datos están correctos?

## Visualización final FR vs PCI vs ED.

Combinar las tres variables: FR, ingresos, educación, en una sola visualización.

### Tratamiento de los datos.

In [ ]:
# Creamos una lista de países que tengan la información de las tres variables.
lista_paises_final3 = []
for p in lista_paises_final:
    if p in lista_paises_final2:
        lista_paises_final3.append(p)

print(len(lista_paises_final3), lista_paises_final3)

In [ ]:
# Ahora creamos tres listas para las variables FR, PCI y educación, para los 
# países de la lista anterior.
lf = []
li = []
le = []
for p in lista_paises_final3:
    lf.append(paises.get_group(p)['Value'].iloc[2])  # El lugar 2 corresponde a los datos para 2010
    li.append(ingreso_pais.get_group(p)['Value'].iloc[0]) # Solo se tiene el dato para 2010
    e = educacion_pais.get_group((p, gender))
    le.append(e['Observation Value'].iloc[0]) # Solo se tiene el dato para 2010

print('\nFR (tamaño: {}) \nDatos :\n{}'.format(len(lf), lf))
print('\nPCI (tamaño: {}) \nDatos :\n{}'.format(len(li), li))
print('\nEducación (tamaño: {}) \nDatos :\n{}'.format(len(le), le))

Hacemos la visualización usando la variable eduación como un valor para el área de los círculos que se van a graficar.

In [ ]:
fig = plt.figure(figsize=(10,6)) # Cambiamos el tamaño de la figura

# Eje x: FR
# Eje y: PCI
# Area y color: Educación

se = np.array(le) * 5
max_se = np.max(se)
color = [int(c) for c in se]
scatter = plt.scatter(lf, li, marker='o', c=se, alpha=0.5, s=se, cmap="viridis") 
plt.colorbar(mappable=scatter, label='Nivel educativo')

plt.title('fuente: http://data.un.org', loc='right', fontstyle='italic', fontsize=10)
plt.suptitle(' FR vs PCI vs Educación ', y = 0.98, fontsize=14)
    
plt.grid(ls='--', lw=0.5, c='#AAAAAA')
plt.xlabel('FR [No. de hijos por mujer]')
plt.ylabel('PCI [USD/Año]')

# Gráficamos el ajuste entre fertilidad e ingresos
plt.plot(x, y, '--', c='red', lw=2.0, zorder=0, alpha=0.75,
         label='Ajuste FR: a={:5.3f}, b={:5.3}, c={:5.3f}'.format(popt[0], popt[1], popt[2]))

# Identificamos algunos países en el gráfico
ipm = lista_paises_final3.index('Mexico')
plt.annotate('México', xy=(lf[ipm], li[ipm]), xytext=(5, 30000),
             bbox=dict(boxstyle='round', facecolor='green', edgecolor='black', alpha=0.75, linewidth=1.1),
             arrowprops=dict(arrowstyle='->', facecolor='black', edgecolor='black'),
             fontsize=10, color='white', horizontalalignment='center')

ipn = lista_paises_final3.index('Niger')
plt.annotate('Nigeria', xy=(lf[ipn], li[ipn]), xytext=(7, 5000),
             bbox=dict(boxstyle='round', facecolor='yellow', edgecolor='black', alpha=0.75, linewidth=1.1),
             arrowprops=dict(arrowstyle='->', facecolor='black', edgecolor='black'),
             fontsize=10, color='black', horizontalalignment='center')

ipj = lista_paises_final3.index('China')
plt.annotate('China', xy=(lf[ipj], li[ipj]), xytext=(1.5, 1000),
             bbox=dict(boxstyle='round', facecolor='red', edgecolor='black', alpha=0.75, linewidth=1.1),
             arrowprops=dict(arrowstyle='->', facecolor='black', edgecolor='black'),
             fontsize=10, color='white', horizontalalignment='center')

# Agregamos un poco más de información
plt.text(6.75, 40000, 'El área de cada círculo \n representa el nivel \n educativo del país',                   
             transform=plt.gca().transData, 
             horizontalalignment='center', color='black',
             bbox=dict(boxstyle='round', facecolor='gray', edgecolor='black', alpha=0.1, linewidth=0.75)) 

# Usamos escala semilogarítmica
plt.yscale('log')
plt.ylim(1e2,1e5)
plt.xlim(0.5,8)
plt.legend(loc='lower left', fontsize=10)
plt.savefig('FR04_fert_ing_educ.pdf')

#### Ejercicio: 
Use otra escala para los ejes y modifique el mapa de color para los círculos.